## Ex 4.1

In [59]:
import time
import tracemalloc
def log_execution (func):
    def wrapper(*args, **kwargs):
        tracemalloc.start()
        start  = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        current, peak = tracemalloc.get_traced_memory()
        print(f"Function: {func.__name__}, Time taken: {end - start} seconds")
        print(f"Function: {func.__name__}, Memory used: {current} bytes, Peak Memory used: {peak} bytes")
        tracemalloc.stop()
        return result
    return wrapper

In [60]:
@log_execution
def find_character (input_string, character):
    for i in range(len(input_string)):
        if input_string[i] == character:
            return True
        else:
            return False

In [61]:
find_character("snfdskfnjsddsjqwieuqweqw", 'a')

Function: find_character, Time taken: 0.0 seconds
Function: find_character, Memory used: 72 bytes, Peak Memory used: 152 bytes


False

## Ex 4.2

In [62]:
def repeat(num_times):
    def decorator(func):
        def wrapper(*args, **kwargs):
            result = []
            for _ in range(num_times):
                result.append(func(*args, **kwargs))

            return result
        return wrapper
    return decorator

In [63]:
@repeat(num_times=5)
def find_character (input_string, character):
    for i in range(len(input_string)):
        if input_string[i] == character:
            return True
        else:
            return False

In [64]:
find_character("ddfdfsdfsd", "a")

[False, False, False, False, False]

## Ex 4.3

### Not using generator

In [22]:

def read_file_no_yield():
    with open("large_log_file.txt", 'r') as file:
        return file.readlines()

In [65]:
@log_execution
def search_error():
    lines = read_file_no_yield()
    for line in lines:
        if "ERROR" in line:
            pass

In [66]:
search_error()

Function: search_error, Time taken: 17.682008266448975 seconds
Function: search_error, Memory used: 149130 bytes, Peak Memory used: 954261774 bytes


### Using a generator

In [67]:

def read_file_with_yield():
    with open("large_log_file.txt") as file:
        for line in file:
            yield line.strip()
@log_execution
def search_error():
    for line in read_file_with_yield():
        if "ERROR" in line:
            pass

In [68]:
search_error()

Function: search_error, Time taken: 13.838605403900146 seconds
Function: search_error, Memory used: 149030 bytes, Peak Memory used: 183207 bytes


## Ex 4.4

- MultiThreading: 	Multiple threads within the same process, sharing memory space.
    - Best for: I/O-bound tasks, such as reading/writing files, network requests, and user interfaces where waiting for external resources is common.
- Multiprocessing: Multiple processes with separate memory spaces, each running its own Python interpreter.
    - Best for: CPU-bound tasks, such as mathematical calculations, image processing, and database operations where parallelism is beneficial.
- Asynchronous Programming: Single-threaded event loop managing tasks via cooperative multitasking (tasks yield control).
    - Best for: I/O-bound tasks like handling large numbers of network connections, APIs, real-time systems, and web scraping.

## Ex 4.5

In [71]:
import os
import random
import aiofiles
import shutil

In [75]:
def create_directory(directory = "files"):
    if os.path.exists(directory):
    # If it exists, delete all files in the directory
        for filename in os.listdir(directory):
            file_path = os.path.join(directory, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)  # Remove the file
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)  # Remove the sub-directory
            except Exception as e:
                print(f"Failed to delete {file_path}. Reason: {e}")
    else:
        # If the directory does not exist, create it
        os.makedirs(directory)

In [80]:
create_directory()

In [81]:
# Step 1: Generate 1000 text files with random numbers of lines
@log_execution
def generate_files(directory = "files"):
    for i in range(1000):
        file_name = f"{directory}/file_{i+1}.txt"
        num_lines = random.randint(1, 500)  # Random number of lines between 1 and 500
        with open(file_name, 'w') as f:
            for _ in range(num_lines):
                f.write("This is a line of text.\n")



In [82]:
generate_files()

Function: generate_files, Time taken: 1.4308009147644043 seconds
Function: generate_files, Memory used: 148596 bytes, Peak Memory used: 190391 bytes


## Ex 4.6

### Not using asynchronisation

In [70]:
# Step 2: Create result.txt to store the number of lines for each file
@log_execution
def create_result_txt(path = "result.txt", directory = "files"):
    with open(path, 'w') as result_file:
        for i in range(1000):
            file_name = f"{directory}/file_{i+1}.txt"
            with open(file_name, 'r') as f:
                lines =  f.readlines() 
                num_lines = len(lines)
            result_file.write(f"{file_name}: {num_lines} lines\n")

In [83]:
create_result_txt()

Function: create_result_txt, Time taken: 4.20612907409668 seconds
Function: create_result_txt, Memory used: 297286 bytes, Peak Memory used: 413516 bytes


### Using asynchronisation

In [51]:
@log_execution
async def create_result_txt_with_async(path="result.txt", directory="files"):
    async with aiofiles.open(path, 'w') as result_file:
        for i in range(1000):
            file_name = f"{directory}/file_{i+1}.txt"
            async with aiofiles.open(file_name, 'r') as f:
                lines = await f.readlines() 
                num_lines = len(lines) 
            await result_file.write(f"{file_name}: {num_lines} lines\n")

In [84]:
await create_result_txt_with_async()

Function: create_result_txt_with_async, Time taken: 0.0 seconds
